## Machine Learning Record Mining

Project to create a pipeline that uses GeoDeepDive's output to find Unaquired Sites for Neotoma.

Using NLP parsed text and a Data Science approach, identify whether a paper is suitable for Neotoma and detect features such as 'Site Name', 'Location', 'Age Span' and 'Site Descriptions'.

In [4]:
# Loading libraries

import numpy as np
import pandas as pd
import csv
import psycopg2
import re

# Load Postgres Server
from src.config import config

In [55]:
# Options for DF display
#pd.set_option('display.max_colwidth', 30)
#pd.set_option('display.max_rows', 10)

pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

## Loading and viewing the Data

### Loading NLP Sentences

In [6]:
# Connect to PostgreSQL server from terminal:
# pg_ctl -D PSQL_Data -l logfile start

try:
    params = config()
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(**params)
    # Create a new cursor
    cur = conn.cursor()
    nlp_sentences = pd.read_sql_query('''SELECT * FROM sentences;''', conn)
    # Close the cursor and connection to so the server can allocate
    # bandwidth to other requests
    cur.close()
    conn.close()
    print('Data obtained from PostgreSQL')
    
# If no SQL db, load from a file
except:
    header_list = ["_gddid", "sentid", "wordidx", "words", "part_of_speech", "special_class", 
               "lemmas", "word_type", "word_modified"]
    nlp_sentences = pd.read_csv("../Do_not_commit_data/sentences_nlp352", sep='\t', names = header_list)
    nlp_sentences = nlp_sentences.replace('"', '', regex = True)\
                                 .replace('\{', '', regex = True)\
                                 .replace('}', '', regex = True)\
                                 .replace(',', ',', regex = True)
    nlp_sentences['wordidx']= nlp_sentences['wordidx'].str.split(",")
    nlp_sentences['words']= nlp_sentences['words'].str.split(",")
    nlp_sentences['poses']= nlp_sentences['poses'].str.split(",")
    nlp_sentences['ners']= nlp_sentences['ners'].str.split(",")
    nlp_sentences['lemmas']= nlp_sentences['lemmas'].str.split(",")
    nlp_sentences['dep_paths']= nlp_sentences['dep_paths'].str.split(",")
    nlp_sentences['dep_parents']= nlp_sentences['dep_parents'].str.split(",")
    print('Data obtained from text file')

Data obtained from PostgreSQL


In [7]:
nlp_sentences.head(10)

,_gddid,sentid,wordidx,words,part_of_speech,special_class,lemmas,word_type,word_modified
0,54b43266e138239d8684efed,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9...","[Available, online, at, ww...","[JJ, NN, IN, NNP, NNP, NNP...","[O, O, O, O, O, O, NUMBER,...","[available, online, at, ww...","[dep, dep, dep, dep, dep, ...","[218, 218, 218, 218, 218, ..."
1,54b43266e138239d8684efed,2,"[1, 2, 3, 4, 5, 6, 7, 8, 9...","[The, Chihuahueños, Bog, r...","[DT, NNP, NN, NN, VBZ, TO,...","[O, O, O, O, O, O, O, NUMB...","[the, Chihuahueños, bog, r...","[det, compound, compound, ...","[4, 4, 4, 5, 0, 11, 11, 11..."
2,54b43266e138239d8684efed,3,"[1, 2, 3, 4, 5, 6, 7, 8, 9...","[An, Artemisia, steppe, ,,...","[DT, NNP, NN, ,, RB, DT, J...","[O, O, O, O, O, O, O, LOCA...","[a, Artemisia, steppe, ,, ...","[det, compound, nsubj, , a...","[3, 3, 10, 0, 9, 9, 9, 9, ..."
3,54b43266e138239d8684efed,4,"[1, 2, 3, 4, 5, 6, 7, 8, 9...","[C/N, ratios, ,, δ13C, and...","[JJ, NNS, ,, NN, CC, NN, N...","[O, O, O, O, O, O, O, O, O...","[c/n, ratio, ,, δ13c, and,...","[amod, compound, , conj:an...","[7, 7, 0, 2, 2, 2, 8, 0, 1..."
4,54b43266e138239d8684efed,5,"[1, 2, 3, 4, 5, 6, 7, 8, 9...","[Higher, percentages, of, ...","[JJR, NNS, IN, JJ, NN, CC,...","[O, O, O, O, O, O, O, O, O...","[higher, percentage, of, a...","[amod, nsubj, case, amod, ...","[2, 10, 5, 5, 2, 5, 9, 9, ..."
5,54b43266e138239d8684efed,294,"[1, 2, 3, 4, 5, 6]","[Anderson, ,, R.S., ,, 198...","[NNP, ,, NNP, ,, CD, .]","[PERSON, O, PERSON, O, DAT...","[Anderson, ,, R.S., ,, 198...","[compound, , , , amod, ]","[3, 0, 0, 0, 3, 0]"
6,54b43266e138239d8684efed,6,"[1, 2, 3, 4, 5, 6, 7, 8, 9...","[From, ca., 8500, to, 6400...","[IN, FW, CD, TO, CD, JJ, N...","[O, O, NUMBER, O, NUMBER, ...","[from, ca., 8500, to, 6400...","[case, nmod:from, compound...","[2, 19, 5, 5, 8, 8, 8, 19,..."
7,54b43266e138239d8684efed,7,"[1, 2, 3, 4, 5, 6, 7, 8, 9...","[C/N, ratios, declined, to...","[JJ, NNS, VBD, TO, PRP$, J...","[O, O, O, O, O, O, O, O, O...","[c/n, ratio, decline, to, ...","[amod, nsubj, , case, nmod...","[2, 3, 0, 8, 8, 8, 8, 3, 0..."
8,54b43266e138239d8684efed,8,"[1, 2, 3, 4, 5, 6, 7, 8, 9...","[Wetter, conditions, retur...","[JJ, NNS, VBD, IN, CD, JJ,...","[O, O, O, O, DATE, O, DURA...","[wetter, condition, return...","[amod, nsubj, , case, numm...","[2, 3, 0, 8, 8, 8, 8, 3, 0..."
9,54b43266e138239d8684efed,9,"[1, 2, 3, 4, 5, 6, 7, 8, 9...","[Higher, charcoal, influx,...","[JJR, NN, NN, NNS, IN, CD,...","[O, O, O, O, O, DATE, O, D...","[higher, charcoal, influx,...","[amod, compound, compound,...","[4, 4, 4, 11, 9, 9, 9, 9, ..."


### Loading Bibliography Data

In [8]:
import json
with open('../Do_not_commit_data/bibjson', 'r') as f:
    bib_dict = json.load(f)

In [9]:
# Normalizing data so that we have access to the 'identifier'

# TODO Load into SQL server and connect through SQL
bibliography = pd.io.json.json_normalize(bib_dict,'identifier',['publisher', 'title', ['journal','name', 'name'], ['author'], 'year', 'number', 'volumne', ['link'], '_gddid', 'type', 'pages'], record_prefix='_', errors='ignore')

bibliography['url'] = bibliography['link'].str.split("'type'", expand = True)[0]

In [10]:
# TODO Flatten 'URL' 

bibliography.head(10)

,_type,_id,publisher,title,journal.name.name,author,year,number,volumne,link,_gddid,type,pages,url
0,doi,10.1016/S0277-3791(99)00007-4,Elsevier,"Palaeoclimate, chronology ...",Quaternary Science Reviews,"[{'name': 'Huntley, B.'}, ...",1999,7,NaN,[{'url': 'http://www.scien...,550453fde1382326932d85f7,article,945--960,NaN
1,doi,10.1139/b92-002,Canadian Science Publishing,Holocene history of forest...,Canadian Journal of Botany,"[{'name': 'Bennett, K. D.'}]",1992,1,NaN,[{'url': 'http://www.nrcre...,578b5aabcf58f1587003efba,article,6--18,NaN
2,doi,10.1016/0031-0182(80)90043-7,Elsevier,Glacial sequence and envir...,"Palaeogeography, Palaeocli...","[{'name': 'Van Der Hammen,...",1980,NaN,NaN,[{'url': 'http://www.scien...,54b43244e138239d868493cd,article,247--340,NaN
3,doi,10.1130/g35541.1,GSA,A record of sustained preh...,Geology,"[{'name': 'Munoz, S. E.'},...",2014,6,NaN,[{'url': 'http://dx.doi.or...,57c5b941cf58f1338eaddb5b,article,499--502,NaN
4,doi,10.1016/0033-5894(81)90128-9,Elsevier,Plant and insect fossils a...,Quaternary Research,"[{'name': 'Ashworth, Allan...",1981,1,NaN,[{'url': 'http://www.scien...,54b43269e138239d8684f87f,article,66--79,NaN
5,doi,10.1080/00173130903435846,Taylor and Francis,10. Na Bahně (Czech Republ...,Grana,"[{'name': 'Pokorný, Petr'}...",2010,1,NaN,[{'url': 'http://www.tandf...,58d27c30cf58f121bb4ff5db,article,79--81,NaN
6,doi,10.1139/e98-095,Canadian Science Publishing,Age verification of the La...,Canadian Journal of Earth ...,"[{'name': 'Lowell, Thomas ...",1999,3,NaN,[{'url': 'http://www.nrcre...,574629c5cf58f15d3f588144,article,383--393,NaN
7,doi,10.1016/j.yqres.2013.03.001,Elsevier,Holocene vegetation histor...,Quaternary Research,"[{'name': 'Lucas, Jennifer...",2013,3,NaN,[{'url': 'http://www.scien...,54b43265e138239d8684ee17,article,366--376,NaN
8,doi,10.1016/j.yqres.2009.05.001,Elsevier,Synchronization of late-gl...,Quaternary Research,"[{'name': 'Gonzales, Leila...",2009,2,NaN,[{'url': 'http://www.scien...,54b43265e138239d8684ef96,article,234--245,NaN
9,doi,10.1002/jqs.979,Wiley,Contrasting pollen histori...,Journal of Quaternary Science,"[{'name': 'Hanselman, Jenn...",2005,7-8,NaN,[{'url': 'http://doi.wiley...,56f8f62ccf58f1710ac939e0,article,663--670,NaN


## EDA

Reviewing our data includes skimming through some papers online and seeing if the data is consistent with our NLP Sentences dataframe.

From there, we can also visualize what we would like our model to predict: 'Location', 'Site Name', 'Age Span', and 'Site Description' from a "Human perspective".

In [11]:
def order_article(article_id):
    '''
    Function to find an article by its gddid in the NLP sentences and have it displayed in order
    
    Keyword arguments:
    article_id -- gddid
    
    Returns:
    article ordered by sentence index
    '''
    article = nlp_sentences[nlp_sentences['_gddid'] == article_id]
    return article[['sentid', 'words']].sort_values(by = 'sentid')

In [12]:
# Article 01 
order_article('550453fde1382326932d85f7')

,sentid,words
94865,1,"[Quaternary, Science, Revi..."
94866,2,"[Plant, macrofossil, data,..."
94867,3,"[Quantitative, palaeoclima..."
94868,4,"[The, chronological, frame..."
94870,5,"[Surface, conditions, in, ..."
...,...,...
95325,448,"[Vegetation, history, and,..."
95326,449,"[Quaternary, Science, Revi..."
95327,450,"[Zolitschka, ,, B., ,, &, ..."
95329,451,"[Sedimentology, ,, dating,..."


Skimmed info: 
 * `Article Name:` Palaeoclimate_chronology_and_vegetation_history_of_the_Weichselian_Lateglacial
 * `Link`: http://www.sciencedirect.com/science/article/pii/S0277379199000074  
 * `Site Name`:  Laghi di Monticchio
 * `Location`:  40° 56' 40" N, 15° 36' 48" E
 * `Age Span`: 
 * `Site Descriptions`: The record from Lago Grande di Monticchio now adds macrofossil evidence of the presence of relatively boreal tree taxa during the Lateglacial interstadial and provides a chronology based upon annually laminated sediments.

In [13]:
# Article 2
order_article('54b43266e138239d8684efed')

,sentid,words
0,1,"[Available, online, at, ww..."
1,2,"[The, Chihuahueños, Bog, r..."
2,3,"[An, Artemisia, steppe, ,,..."
3,4,"[C/N, ratios, ,, δ13C, and..."
4,5,"[Higher, percentages, of, ..."
...,...,...
605,572,"[Is, the, Valles, caldera,..."
144,573,"[Geology, 23, ,, 411, --, ..."
607,574,"[Wright, Jr., ,, H.E., ,, ..."
608,575,"[Present, and, past, veget..."


Skimmed info:  
* `Article:` Development of the Mixed Connifer Forest in Northern New Mexico and its relationship to Holocene environmental change
* `Link`: http://www.sciencedirect.com/science/article/pii/S0033589407001512
* `Site Name`:  Chihuahuenos Bog 
* `Location`:  ??? No Given Coordinates 
* `Age Span`:   
* `Site Descriptions`:  Chihuahueños Bog is located within the mixed conifer forest and has affinities to the flora of the southern Rocky Mountains(Foxx and Tierney, 1985; Allen, 2004). Although the area immediately surrounding the bog has been heavily logged in recent years, nearly every conifer species growing in montaneforests of the Jemez mountains is represented either along theshore of the bog or in the surrounding uplands.  

In [14]:
# Article 3
order_article('57c5b941cf58f1338eaddb5b')

,sentid,words
67708,1,"[A, record, of, sustained,..."
67709,2,"[Here, we, report, a, high..."
67710,3,"[Palynological, and, carbo..."
67712,4,"[Rapid, forest, clearance,..."
67713,5,"[Agricultural, intensiﬁcat..."
...,...,...
67845,123,"[Simon, ,, M.L., ,, and, P..."
67846,124,"[Smith, ,, B.D., ,, and, Y..."
67848,125,"[Sugita, ,, S., ,, 1993, ,..."
67849,126,"[Trubitt, ,, M.B.D., ,, 20..."


Skimmed info:  
* `Link`: http://dx.doi.org/10.1130/g35541.1  # No Full access to article
* `Site Name`:  Cahokia region, Illinois, USA
* `Location`:  N38°42'00" - N38°42'00", W90°04'60" - W90°04'60" - Not found on text (yet).
* `Age Span`:   
* `Site Descriptions`: Record from Horseshoe Lake, an oxbow lake in the central Mississippi River valley that is adjacent to the Cahokia site (Illinois, USA), the largest prehistoric settlement north of Mexico.   

In [15]:
# Article 4
order_article('58d29193cf58f14928755ba5')

,sentid,words
110659,1,"[Grana, ISSN, :, 0017-3134..."
110660,2,"[Peat-bog, Begbunar, -LRB-..."
110661,3,"[Peat-bog, Begbunar, -LRB-..."
110662,4,"[Submit, your, article, to..."
110665,5,"[Peat-bog, Begbunar, -LRB-..."
...,...,...
110740,80,"[Ann, .]"
110741,81,"[Sofia, Univ., .]"
110742,82,"[Fac, .]"
110743,83,"[Geol, .]"


In [16]:
# Try to see the exact location. Wont show last few words.
pd.DataFrame(order_article('58d29193cf58f14928755ba5').iloc[4])

,110665
sentid,5
words,"[Peat-bog, Begbunar, -LRB-..."


Skimmed info:  
* `Link:`http://www.tandfonline.com/doi/abs/10.1080/00173130902965157
* `Site Name`:  Begbunar
* `Location`:  42°09′ N, 22° 33′ E; 1750 ; in SQL file shows as: `42,°,09,cents,N,,,22,°,33,cents,E`
* `Age Span`:   
* `Site Descriptions`: Located in the central treeless zone of the Osogovo mountains, which are situated at the border between south-western Bulgaria and the north-eastern former Yugoslav Republic of Macedonia.

In [17]:
# Article 5
order_article('57928e07cf58f133d1c26609')

,sentid,words
39606,1,"[Timberline, fluctuations,..."
39607,2,"[By, tracking, climaticall..."
39608,3,"[Pollen, data, suggest, th..."
39609,4,"[The, inferred, climate, w..."
39611,5,"[Abies, -LRB-, fir, -RRB-,..."
...,...,...
40107,448,"[Weber, ,, W., A., ,, 1987..."
40108,449,"[Whitlock, ,, C., ,, 1993,..."
40110,450,"[Whitlock, ,, C., ,, and, ..."
40111,451,"[Wright, ,, H., E., ,, Jr...."


In [18]:
# Print the exact line where locations are.
pd.DataFrame(order_article('57928e07cf58f133d1c26609').iloc[179])

,39801
sentid,180
words,"[LOCATION, AND, AGE, OF, F..."


In [52]:
other_example = nlp_sentences[nlp_sentences['_gddid'] == '57928e07cf58f133d1c26609']
other_example = other_example[other_example['sentid'] == 180]
other_example[['words', 'liststring', 'dms_regex']]

,words,liststring,dms_regex
39801,"[LOCATION, AND, AGE, OF, FOSSIL, STUDY, SITES, Core, Location, Lat, Long, -LRB-, °, N, -RRB-, -LRB-, °, W, -RRB-, Elevation, Depth, -LRB-, m, -RRB-, -LRB-, cm, -RRB-, Age, Laboratory, -LRB-, yr, B.P., -RRB-, number, Cottonwood, III, 38, °, 49, ′, 50, ′, ′, 106, °, 24, ′, 45, ′, ′, 3670, 40, --, 50, 80, --, 90, 120, --, 130, 2460, ±, 130, 5790, ±, 180, 8230, ±, 200, A-4481, A-4482, A-4483, Cottonwood, IV, 38, °, 49, ′, 50, ′, ′, 106, °, 24, ′, 45, ′, ′, 3670, 60, --, 80, 5610, ±, 150, A-4487, 148, --, 168, 8790, ...]","LOCATION,AND,AGE,OF,FOSSIL,STUDY,SITES,Core,Location,Lat,Long,-LRB-,°,N,-RRB-,-LRB-,°,W,-RRB-,Elevation,Depth,-LRB-,m,-RRB-,-LRB-,cm,-RRB-,Age,Laboratory,-LRB-,yr,B.P.,-RRB-,number,Cottonwood,III,38,°,49,′,50,′,′,106,°,24,′,45,′,′,3670,40,--,50,80,--,90,120,--,130,2460,±,130,5790,±,180,8230,±,200,A-4481,A-4482,A-4483,Cottonwood,IV,38,°,49,′,50,′,′,106,°,24,′,45,′,′,3670,60,--,80,5610,±,150,A-4487,148,--,168,8790,±,230,A-4488,Red,Lady,I,38,°,52,′,50,′,′,107,°,2,′,30,′,′,3350,75,--,95,4675,±,155,GX-8350,Red,Lady,II,38,°,52,′,50,′,′,107,°,2,′,30,′,′,3350,10,--,20,45,--,55,74,--,84,100,±,70,2520,±,90,4680,±,120,A-4484,A-4485,A-4486,Red,Well,38,°,53,′,40,′,′,107,°,3,′,15,′,′,3290,80,--,100,2805,±,160,GX-8351,Copley,II,38,°,52,′,28,′,′,107,°,05,′,3250,25,--,35,62,--,72,102,--,112,125,--,135,155,--,165,181,--,191,212,--,222,1570,±,150,5690,±,150,6900,±,220,8210,±,355,10325,±,465,12810,±,750,15370,±,880,A-4479,A-4480,GX-7906,GX-12995,GX-12996,GX-12997,GX-7907,Splains,II,38,°,50,′,107,°,4,′,30,′,′,3160,45,--,55,90,--,100,190,--,200,266,--,286,291,--,292,3880,±,170,5040,±,140,8750,±,220,11040,±,240,12020,±,530,A-4408,A-4407,A-4409,A-2535,AA-642,Splains,Gulch,I,38,°,50,′,107,°,4,′,30,′,′,3150,34,--,48,80,--,98,134,--,148,190,--,200,4440,±,120,5620,±,140,7340,±,240,8560,±,600,A-4410,A-2534,A-4411,A-2424,Ironbog,38,°,52,′,107,°,2,′,30,′,′,2920,48,--,50,58,--,66,134,--,136,170,195,--,200,218,--,220,2640,±,90,3105,±,155,6300,±,80,7700,±,70,7100,±,255,8260,±,80,AA-1125,GX-7905,AA-1126,AA-1127,GX-7339,AA-1128,Alkali,Basin,I,38,°,45,′,106,°,50,′,2750,137,3480,±,135,GX-6796,300,7857,±,260,GX-6795,410,12975,±,430,GX-6794,Alkali,Basin,II,38,°,45,′,106,°,50,′,2750,533,--,549,17440,±,1200,A-2421,conifers,,,Picea,and,Abies,,,drop,to,levels,found,above,treeline,in,the,krummholz,zone,today,,,and,Pinus,pollen,percentages,and,accumulation,increase,-LRB-,see,Figs.,6A,and,7A,-RRB-,.","[38,°,49,′,50,′,′,106,°,24,′,45,′,′,, 38,°,49,′,50,′,′,106,°,24,′,45,′,′,, 38,°,52,′,50,′,′,107,°,2,′,30,′,′,, 38,°,52,′,50,′,′,107,°,2,′,30,′,′,, 38,°,53,′,40,′,′,107,°,3,′,15,′,′,, 38,°,52,′,28,′,′,107,°,05,′,, 38,°,50,′,107,°,4,′,30,′,′,, 38,°,50,′,107,°,4,′,30,′,′,, 38,°,52,′,107,°,2,′,30,′,′,, 38,°,45,′,106,°,50,′,, 38,°,45,′,106,°,50,′,]"


Skimmed info:  
* `Link`: http://dx.doi.org/10.1130/0016-7606(1997)109<1306:tfalqp>2.3.co
* `Site Name`:  Cottonwood III, Cottonwood IV, Red Lady, Red Well, Splains
* `Location`:  38°49'50' 106°24'45', 38°49'50' 106°24'45', 38°52'50' 107°2'30', 38°52'50' 107°2'30',38°53'40' 107°3'15', 38°52'28' 107°05', 38°50' 107°4'30', 38°50' 107°4'30', 38°52' 107°2'30', 38°45' 106°50', 38°45' 106°50'
* `Age Span`:   17000 - 400
* `Site Descriptions`:   The lower timberline or lower forest border is the lower elevational limit of a continuous forest belt. In the Rocky Mountains it forms a sharp contrast with shrublands or grasslands in valley bottoms. Trees may extend below the lower timberline in favorable habitats, along streams or on rocky outcrops.  

In [20]:
# Find the article in the bibliography by gddid to retrieve Article Name

bibliography[bibliography['_gddid'] == '57928e07cf58f133d1c26609']

,_type,_id,publisher,title,journal.name.name,author,year,number,volumne,link,_gddid,type,pages,url
93,doi,10.1130/0016-7606(1997)109...,GSA,Timberline fluctuations an...,Geological Society of Amer...,"[{'name': 'Fall, Patricia ...",1997,10,NaN,[{'url': 'http://dx.doi.or...,57928e07cf58f133d1c26609,article,1306--1320,NaN


### Finding Latitude and Longitud via REGEX 

After reviewing some PDF's and viewing the data parsing, I noticed some possible patterns for coordinates.

<img src="figures/img/00_snippet_of_pdf.png" alt="Drawing" style="width: 300px;"/>

Which translates to our pandas dataframe as:

![alt text](figures/img/01_snippet_in_sql.png "PandasVersion")

As a first problem, we can notice that the degrees symbol `°` was translated as a `3`. On some other occassions, it is confused as an `o` or a middle point `◦`. For `'`, this symbol got confused by the word `cents`.

I am still going to use normal REGEX to find coordinates in the most intuitive way. 

A coordinate for longitude is a number ranging from -180° to 180°. It has three components, `°, ',"` and it covers the position in E or W.

Latitude goes from -90° to 90°, has the same three components and goes from S to N.

Using REGEX, and assuming data consistency, the following code should be able to extract coordinates. 

In [44]:
dms_regex = r"([-]?\d{1,3}[°|,|′|\']{1,3}\d{1,2}[,|′|\']{1,3}\d{0,2}[,|′|\']{1,4}[NESWnesw]?[\s|,|\']+?[-]?\d{1,3}[°|,|′|\']+\d{1,2}[,|′|\']+\d{0,2}[,|′|\'][,|′|\']{0,4}[NESWnesw]?)"

dd_regex = "[\\{,][-]?[1]?[0-9]{1,2}\\.[0-9]{1,}[,]?[NESWnesw],"

In [45]:
# TODO Find REGEX of couples of numbers (123, 234)
nlp_sentences['liststring'] = nlp_sentences['words'].apply(lambda x: ','.join(map(str, x)))
nlp_sentences['dms_regex'] = nlp_sentences['liststring'].str.findall(dms_regex)


#nlp_sentences[['dms_regex', 'words']]

nlp_sentences[['liststring', 'words', 'dms_regex']].head(3)

,liststring,words,dms_regex
0,"Available,online,at,www.sciencedirect.com,Quaternary,Research,69,-LRB-,2008,-RRB-,263,--,275,www.elsevier.com/locate/yqres,Development,of,the,mixed,conifer,forest,in,northern,New,Mexico,and,its,relationship,to,Holocene,environmental,change,R.,Scott,Anderson,a,,,b,,,,,Renata,B.,Jass,b,,1,,,Jaime,L.,Toney,b,,2,,,Craig,D.,Allen,c,,,Luz,M.,Cisneros-Dozal,d,,,Marcey,Hess,d,,,Jeff,Heikoop,d,,,Julianna,Fessenden,d,a,Center,for,Environmental,Sciences,&,Education,,,Box,5694,,,Northern,Arizona,University,,,Flagstaff,,,AZ,86011,,,USA,b,Quaternary,Sciences,Program,&,Bilby,Research,Center,,,Box,6013,,,Northern,Arizona,University,,,Flagstaff,,,AZ,86011,,,USA,c,U.S.,Geological,Survey,,,Jemez,Mountains,Field,Station,,,Bandelier,National,Monument,,,HCR-1,,,Box,1,,,#,15,,,Los,Alamos,,,NM,87544,,,USA,d,Earth,&,Environmental,Sciences,Division,,,Hydrology,,,Geochemistry,&,Geology,Group,,,EES-6,,,MS-D462,,,Los,Alamos,National,Laboratory,,,Los,Alamos,,,NM,87545,,,USA,Received,28,April,2007,Available,online,29,January,2008,Abstract,Chihuahueños,Bog,-LRB-,2925,m,-RRB-,in,the,Jemez,Mountains,of,northern,New,Mexico,contains,one,of,the,few,records,of,late-glacial,and,postglacial,development,of,the,mixed,conifer,forest,in,southwestern,North,America,.","[Available, online, at, www.sciencedirect.com, Quaternary, Research, 69, -LRB-, 2008, -RRB-, 263, --, 275, www.elsevier.com/locate/yqres, Development, of, the, mixed, conifer, forest, in, northern, New, Mexico, and, its, relationship, to, Holocene, environmental, change, R., Scott, Anderson, a, ,, b, ,, ,, Renata, B., Jass, b, ,1, ,, Jaime, L., Toney, b, ,2, ,, Craig, D., Allen, c, ,, Luz, M., Cisneros-Dozal, d, ,, Marcey, Hess, d, ,, Jeff, Heikoop, d, ,, Julianna, Fessenden, d, a, Center, for, Environmental, Sciences, &, Education, ,, Box, 5694, ,, Northern, Arizona, University, ,, Flagstaff, ,, AZ, 86011, ,, USA, b, Quaternary, Sciences, Program, &, Bilby, Research, ...]",[]
1,"The,Chihuahueños,Bog,record,extends,to,over,15,000,cal,yr,BP,.","[The, Chihuahueños, Bog, record, extends, to, over, 15,000, cal, yr, BP, .]",[]
2,"An,Artemisia,steppe,,,then,an,open,Picea,woodland,grew,around,a,small,pond,until,ca.,11,700,cal,yr,BP,when,Pinus,ponderosa,became,established,.","[An, Artemisia, steppe, ,, then, an, open, Picea, woodland, grew, around, a, small, pond, until, ca., 11,700, cal, yr, BP, when, Pinus, ponderosa, became, established, .]",[]


In [57]:
italy_example = nlp_sentences[nlp_sentences['_gddid'] == '550453fde1382326932d85f7']
italy_example = italy_example[italy_example['sentid'] == 10]
italy_example[['words', 'liststring', 'dms_regex']]

,words,liststring,dms_regex
94875,"[Introduction, Laghi, di, Monticchio, are, two, maar, lakes, located, in, the, explosion, crater, that, lies, west, of, the, peak, of, Monte, Vulture, ,, near, Mel, '', in, the, Basilicata, region, of, southern, Italy, -LRB-, 403, 56, 40, N, ,, 153, 36, 48, E, -RRB-, -LRB-, Fig., 1, -RRB-, .]","Introduction,Laghi,di,Monticchio,are,two,maar,lakes,located,in,the,explosion,crater,that,lies,west,of,the,peak,of,Monte,Vulture,,,near,Mel,'',in,the,Basilicata,region,of,southern,Italy,-LRB-,403,56,40,N,,,153,36,48,E,-RRB-,-LRB-,Fig.,1,-RRB-,.","[403,56,40,N,,,153,36,48,E]"


In [47]:
italy_example['words'].astype(str).str.len()

94875    391
Name: words, dtype: int64

### Problems with the REGEX approach

As seen before, extracting information using REGEX is very complicated. We would require a lot of queries that would retrieve each particular case.

A possible alternative could be using NER, Named Entity Recognition.

## Linking Neotoma DB to extract locations

In [48]:
neotoma = pd.read_csv("../Do_not_commit_data/data-1590729612420.csv")
neotoma = neotoma[['siteid', 'sitename', 'longitudeeast', 'latitudenorth', 'longitudewest', 'latitudesouth', 'sitedescription', 'doi']]
#neotoma.drop_duplicates(subset ="sitename", keep = False, inplace = True) 
neotoma.head(3)

,siteid,sitename,longitudeeast,latitudenorth,longitudewest,latitudesouth,sitedescription,doi
0,10330,Lac du Sommet,-70.66468,47.71662,-70.66573,47.71382,"The small shallow Lac du Sommet (0.02 km2, 4 m maximum depth, elevation of 830 m a.s.l., 47°43′N, 70°40′W) is located in the boreal forest north of the St Lawrence Estuary (Figure 1). It is of glacial origin and situated on granitic-gneissic bedrock of the Canadian Precambrian Shield in the Laurentian Mountains.",10.1177/0959683611400199
1,10330,Lac du Sommet,-70.66468,47.71662,-70.66573,47.71382,"The small shallow Lac du Sommet (0.02 km2, 4 m maximum depth, elevation of 830 m a.s.l., 47°43′N, 70°40′W) is located in the boreal forest north of the St Lawrence Estuary (Figure 1). It is of glacial origin and situated on granitic-gneissic bedrock of the Canadian Precambrian Shield in the Laurentian Mountains.",10.1177/0959683611400199
2,1729,Myrtle Lake,-93.37853,47.98645,-93.39207,47.97876,"Lake surrounded by peatland. Physiography: Red Lake lowlands. Surrounding vegetation: Sphagnum, Piceto-Chamaedophnetum.",10.1139/b68-190


In [49]:
df = nlp_sentences.merge(bibliography, on = '_gddid')

In [54]:
df[['liststring', 'journal.name.name', 'dms_regex']].head(100)

,liststring,journal.name.name,dms_regex
3314,"Lake,Hope,Simpson,-LRB-,52,',27,',N,,,56,',26,',W,,,elevation,295,m,-RRB-,lies,20,km,southwest,of,Port,Hope,Simpson,in,a,steep-sided,eastwest-trending,valley,that,is,drained,by,the,St.,Lewis,River,.",Canadian Journal of Botany,"[52,',27,',N,,,56,',26,',W]"
3318,"The,lake,basin,itself,has,an,irregular,though,conical,bathymetry,,,with,a,mean,depth,of,3.5,m,and,a,maximum,depth,of,12.5,m.,Independence,Lake,-LRB-,52,',33,',N,,,56,',54,',W,,,elevation,172,m,-RRB-,is,located,in,similar,terrain,20,km,farther,west,along,the,Alexis,River,.",Canadian Journal of Botany,"[52,',33,',N,,,56,',54,',W]"
3320,"Moraine,Lake,-LRB-,52,',16,',N,,,58,',03,',W,,,elevation,385,m,-RRB-,occupies,a,small,-LRB-,45,ha,-RRB-,ice-block,depression,in,the,hummocky,Paradise,Moraine,.",Canadian Journal of Botany,"[52,',16,',N,,,58,',03,',W]"
3324,"St.,Paul,Lake,-LRB-,52,',05,',N,,,57,',50,',W,,,elevation,319,m,-RRB-,is,located,about,20,km,southeast,of,Moraine,Lake,in,the,till-mantled,hills,above,the,St.,Paul,River,.",Canadian Journal of Botany,"[52,',05,',N,,,57,',50,',W]"
3408,"Diagram,of,pollen,percentages,-LRB-,above,-RRB-,and,influx,-LRB-,below,-RRB-,from,the,Moraine,Lake,-LRB-,Labrador,,,52,',16,',N,,,58,',03,',W,-RRB-,core,,,including,lithology,-LRB-,LITH,.,-RRB-",Canadian Journal of Botany,"[52,',16,',N,,,58,',03,',W]"
...,...,...,...
105004,"Introduction,Laghi,di,Monticchio,are,two,maar,lakes,located,in,the,explosion,crater,that,lies,west,of,the,peak,of,Monte,Vulture,,,near,Mel,'',in,the,Basilicata,region,of,southern,Italy,-LRB-,403,56,40,N,,,153,36,48,E,-RRB-,-LRB-,Fig.,1,-RRB-,.",Quaternary Science Reviews,"[403,56,40,N,,,153,36,48,E]"
107854,"Within,the,basin,,,members,of,the,DeForest,Formation,206,0,P.T.,Work,et,al.,/,Palaeogeography,,,Palaeoclimatology,,,Palaeoecology,223,-LRB-,2005,-RRB-,204,--,221,5,Kilometers,10,90,0,50,',nicon,Wapsipi,410,45,',River,410,45,',N,410,40,',Lilienthal,site,Minnesota,Creek,Mud,90,0,50,',Hickory,Creek,Roberts,Creek,Wisconsin,ICNC,Mud,Creek,90,0,45,',90,0,40,',90,0,35,',410,40,',90,0,35,',C,Iowa,Missouri,B,A,Fig.,1,.","Palaeogeography, Palaeoclimatology, Palaeoecology","[410,45,',N,410,40,',, 90,0,45,',90,0,40,',, 90,0,35,',410,40,',90,]"
109253,"Bob,Black,Pond,and,Quicksand,Pond,in,Bartow,County,-LRB-,Watts,,,1970,-RRB-,yield,evidence,that,a,forest,of,Pinus,banksiana,-LRB-,jack,pine,-RRB-,a,n,d,Picea,-LRB-,spruce,-RRB-,was,present,locally,f,r,o,m,2,2,,,9,0,0,to,13,500,yr,ago,.",Geol Soc America Bull,"[2,2,,,9,0,0,]"
109480,"Nothofagus,antarctica,,,Pilgerodendron,uviferum,and,Fitzroya,cupressoides,stands,are,present,710,26,',W,410,44,'S,Serrucho,Wetland,x,0,5,km,Fig.,2,.","Palaeogeography, Palaeoclimatology, Palaeoecology","[710,26,',W,410,44,'S]"


In [56]:
# Filter df to just frames that has REGEX longer than one
mask = (df['dms_regex'].str.len() > 0)
df = df.loc[mask]
df[['words','dms_regex']]

,words,dms_regex
3314,"[Lake, Hope, Simpson, -LRB-, 52, ', 27, ', N, ,, 56, ', 26, ', W, ,, elevation, 295, m, -RRB-, lies, 20, km, southwest, of, Port, Hope, Simpson, in, a, steep-sided, eastwest-trending, valley, that, is, drained, by, the, St., Lewis, River, .]","[52,',27,',N,,,56,',26,',W]"
3318,"[The, lake, basin, itself, has, an, irregular, though, conical, bathymetry, ,, with, a, mean, depth, of, 3.5, m, and, a, maximum, depth, of, 12.5, m., Independence, Lake, -LRB-, 52, ', 33, ', N, ,, 56, ', 54, ', W, ,, elevation, 172, m, -RRB-, is, located, in, similar, terrain, 20, km, farther, west, along, the, Alexis, River, .]","[52,',33,',N,,,56,',54,',W]"
3320,"[Moraine, Lake, -LRB-, 52, ', 16, ', N, ,, 58, ', 03, ', W, ,, elevation, 385, m, -RRB-, occupies, a, small, -LRB-, 45, ha, -RRB-, ice-block, depression, in, the, hummocky, Paradise, Moraine, .]","[52,',16,',N,,,58,',03,',W]"
3324,"[St., Paul, Lake, -LRB-, 52, ', 05, ', N, ,, 57, ', 50, ', W, ,, elevation, 319, m, -RRB-, is, located, about, 20, km, southeast, of, Moraine, Lake, in, the, till-mantled, hills, above, the, St., Paul, River, .]","[52,',05,',N,,,57,',50,',W]"
3408,"[Diagram, of, pollen, percentages, -LRB-, above, -RRB-, and, influx, -LRB-, below, -RRB-, from, the, Moraine, Lake, -LRB-, Labrador, ,, 52, ', 16, ', N, ,, 58, ', 03, ', W, -RRB-, core, ,, including, lithology, -LRB-, LITH, ., -RRB-]","[52,',16,',N,,,58,',03,',W]"
3608,"[Lake, Hope, Simpson, -LRB-, Labrador, ,, 52, ', 27, ', N, ,, 56, ', 26, ', W, -RRB-, sediment, chemistry, diagram, .]","[52,',27,',N,,,56,',26,',W]"
3629,"[Moraine, Lake, -LRB-, Labrador, ,, 52, ', 16, ', N, ,, 58, ', 03, ', W, -RRB-, sediment, chemistry, diagram, .]","[52,',16,',N,,,58,',03,',W]"
5581,"[10, 2, 0, 3, 0, 4, 0, 5, 0, 6, 0, 7, 0, 8, 0, 9, 0, ,, n, 1, 0, 2, 0, 3, 0, 1000, 2000, 3000, 4000, 1000, I, |, ', L, r, rf, i, i, i, __, r, |, k, •, I, •, r, __, _, _, -, ,, -LSB-, -, '', i, -LSB-, --, _, _, i, im, -, /, ~, I, --, k, •, |, v, !]","[0,9,0,,,n,1,0,2,]"
5783,"[In, the, section, Laguna, de, Palacio, Tetraploa, is, present, in, the, samples, 74, ,, 73, ,, 9, ,, 8, ,, 7, and, 1, .]","[74,,,73,,,9,,,8,,,7,]"
5962,"[,, ~, C, 1, 2, 3, '', 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ?]","[1,2,3,'',7,8,9,]"
